<h2>Introduction</h2>

Here you will find a step by step guide to downloading, configuring, and running the Einstein Toolkit. You may use this tutorial on a workstation or laptop, or on a supported cluster. Configuring the Einstein Toolkit on an unsupported cluster is beyond the scope of this tutorial. If you find something that does not work, please feel free to mail users@einsteintoolkit.org.

<h2>Prerequisites</h2>
When using the Einstein Toolkit on a laptop or workstation you will want a number of packages installed in order to download, compile and use the Einstein Toolkit components. If this is a machine which you control (i.e. you have root), you can install using one of the recipes that follow:

On Mac, please first 
- Install [Xcode](https://itunes.apple.com/us/app/xcode/id497799835) from the Apple [App Store](https://itunes.apple.com/us/app/xcode/id497799835). In *addition* agree to Xcode license in Terminal 
  ```bash
  sudo xcodebuild -license
  ```
- install MacPorts for your version of the Mac operating system, if you did not already install it (https://www.macports.org/install.php). 
- Next, please install the following packages, using the commands:
```bash
sudo port -N install pkgconfig gcc8 openmpi fftw-3 gsl jpeg zlib hdf5 +fortran +gfortran openssl ld64 +ld64_xcode
sudo port select mpi openmpi-mp-fortran
```

On Debian/Ubuntu/Mint use this command:
```bash
sudo apt-get install -y subversion gcc git numactl libgsl-dev libpapi-dev python libhwloc-dev make libopenmpi-dev libhdf5-openmpi-dev libfftw3-dev libssl-dev liblapack-dev g++ curl gfortran patch pkg-config libhdf5-dev libjpeg-turbo?-dev
```

On Fedora use this command:
```bash
sudo dnf install -y libjpeg-turbo-devel gcc git lapack-devel make subversion gcc-c++ which papi-devel python hwloc-devel openmpi-devel hdf5-openmpi-devel openssl-devel libtool-ltdl-devel numactl-devel gcc-gfortran findutils hdf5-devel fftw-devel patch gsl-devel pkgconfig
module load mpi/openmpi-x86_64
```
You will have to repeat the `module load` command each time you would like to compile or run the code.

On Centos use this command:
```bash
sudo yum install -y epel-release
sudo yum install -y libjpeg-turbo-devel gcc git lapack-devel make subversion gcc-c++ which papi-devel hwloc-devel openmpi-devel hdf5-openmpi-devel openssl-devel libtool-ltdl-devel numactl-devel gcc-gfortran hdf5-devel fftw-devel patch gsl-devel
```

On OpenSuse use this command:
```bash
sudo zypper install -y curl gcc git lapack-devel make subversion gcc-c++ which papi-devel hwloc-devel openmpi-devel libopenssl-devel libnuma-devel gcc-fortran hdf5-devel libfftw3-3 patch gsl-devel pkg-config
```

<h2>Download</h2>

A script called GetComponents is used to fetch the components of the Einstein Toolkit. GetComponents serves as convenient wrapper around lower level tools like git and svn to download the codes that make up the Einstein toolkit from their individual repositories. You may download and make it executable as follows:

<b>Note:</b> By default, the cells in this notebook are Python 3 commands. However, lines that begin with the ! character are run inside a bash shell. Bash commands to set environment variables or change the working directory will not work if executed with ! (their effects are forgotten immediately). Therefore, when setting the directory a special magic %cd command is used. If you wish to run these commands outside the notebook and in a bash shell, cut and paste only the characters following the initial ! or %.


In [ ]:
%cd ~/

In [ ]:
!curl -kLO https://raw.githubusercontent.com/gridaphobe/CRL/ET_2018_09/GetComponents
!chmod a+x GetComponents

GetComponents accepts a thorn list as an argument. To check out the needed components:

In [ ]:
!./GetComponents --parallel https://bitbucket.org/einsteintoolkit/manifest/raw/ET_2018_09/einsteintoolkit.th

In [ ]:
%cd ~/Cactus

<h2>Configure and build</h2>

The recommended way to compile the Einstein Toolkit is to use the Simulation Factory ("SimFactory").
<h3>Configuring SimFactory for your machine</h3>

The ET depends on various libraries, and needs to interact with machine-specific queueing systems and MPI implementations. As such, it needs to be configured for a given machine. For this, it uses SimFactory. Generally, configuring SimFactory means providing an optionlist, for specifying library locations and build options, a submit script for using the batch queueing system, and a runscript, for specifying how Cactus should be run, e.g. which mpirun command to use.

In [ ]:
!./simfactory/bin/sim setup-silent
# If you are on Mac, uncomment and use the next line instead:
# !./simfactory/bin/sim setup-silent --optionlist osx-macports.cfg --runscript osx-macports.run

After this step is complete you will find your machine's default setup under ./simfactory/mdb/machines/&lt;hostname &gt;.ini
You can edit some of these settings freely, such as "description", "basedir" etc. Some entry edits could result in simulation start-up warnings and/or errors such as "ppn" (processor-per-node meaning number of cores on your machine), "num-threads" (number of threads per core) so such edits must be done with some care.

<h2>Building the Einstein Toolkit</h2>

Assuming that SimFactory has been successfully set up on your machine, you should be able to build the Einstein Toolkit with the command below. The option "--mdbkey make 'make -j2'" sets the make command that will be used by the script. The number used is the number of processors used when building. Even in parallel, this step may take a while, as it compiles all the thorns specified in manifest/einsteintoolkit.th.

Note that the "cat" command on the end of the line is to prevent problems with the display in Jupyter when output comes from multiple threads.

Note: Using too many threads to compile on the test machine may result in compiler failures.

In [ ]:
!./simfactory/bin/sim build --mdbkey make 'make -j1' --thornlist ../einsteintoolkit.th | cat

<h2>Running a simple example</h2>

You can now run the Einstein Toolkit with a simple test parameter file.

In [ ]:
!./simfactory/bin/sim create-submit helloworld \
    --parfile arrangements/CactusExamples/HelloWorld/par/HelloWorld.par --walltime 0:5:0

The above command will submit the simulation to the queue naming it "helloworld" and ask for a 5 minutes long job time, if you are running on a cluster, or run it immediately in the background if you are on a personal laptop or workstation. You can check the status of the simulation with the command below. You can run this command repeatedly until the job shows <pre>[ACTIVE (FINISHED)...</pre> as it's state. Prior to that, it may show up as QUEUED or RUNNING.

In [ ]:
!simfactory/bin/sim list-simulations helloworld

Once it finished you can look at the output with the command below.

In [ ]:
!simfactory/bin/sim show-output helloworld 

If you see <pre>INFO (HelloWorld): Hello World!</pre> anywhere in the above output, then congratulations, you have successfully downloaded, compiled and run the Einstein Toolkit! You may now want to try some of the other tutorials to explore some interesting physics examples.

<h2>Running single star simulation</h2>

What follows is the much more computationally intensive example of simulating a static TOV star. We will first use two sed commands to strip this down from a production quality run to something more manageable. Note that we use 2 procs for this simulation, which may be more than your machine supports. If that is the case, please adjust procs and ppn-used accordingly.

In [ ]:
# modify parameter file for smaller memory footprint using sed by changing CoordBase::dx from 8 to 12
!sed '/CoordBase::d[xyz]/s/8/12/' < par/static_tov.par > par/static_tov_small.par
# Shorten the run time to 250
!sed '/cctk_final_time/s/1000/250/' <  par/static_tov_small.par >  par/static_tov_small_short.par

In [ ]:
# start simulation, watch log output
!./simfactory/bin/sim create-run static_tov \
  --parfile=par/static_tov_small_short.par --procs=2 --num-threads=1 --ppn-used=2  --walltime=8:0:0 | cat

<h2>Plotting the Output</h2>

The following commands will generate a simple line plot of the data. They will work in a python script as easily as they do in the notebook (just remove the "%matplotlib inline" directive).

In [ ]:
# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

Numpy has a routine called genfromtxt() which is an extremely efficient reader of textual arrays of floating point numbers. This is well-suited to Cactus .asc files.

In [ ]:
import os
home = os.environ["HOME"]
lin_data = np.genfromtxt(home+"/simulations/static_tov/output-0000/static_tov_small_short/hydrobase-rho.maximum.asc")

This is all you need to do to plot the data once you've loaded it. Note, this uses Python array notation to grab columns 1 and 2 of the data file.

In [ ]:
plt.plot(lin_data[:,1],lin_data[:,2])